<a href="https://colab.research.google.com/github/KamonohashiPerry/PyTorch/blob/master/PyTorch_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorchのインストール
+ Colabの編集→ノートブックの設定でGPUモードにすることをお忘れなく。

In [0]:
# pip install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 484.0MB 89.0MB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.4.0 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1


In [0]:
# pip install torchvision

     |████████████████████████████████| 734.6MB 23kB/s 
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successfully uninstalled torch-0.4.0


In [0]:
# pip install tqdm

インストールできたかの確認

In [0]:
import torch

In [2]:
torch.tensor([1, 2, 3]).to("cuda:0")

tensor([1, 2, 3], device='cuda:0')

In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse

# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# # drive mean root directory of  google drive
# !mkdir -p drive
# !google-drive-ocamlfuse drive
# !ls drive/"Colab Notebooks"/PyTorch

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

import warnings
warnings.filterwarnings('ignore')

In [6]:
!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip

--2019-12-28 06:08:32--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  56.8MB/s    in 3.5s    

2019-12-28 06:08:36 (53.9 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [7]:
!unzip ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [8]:
pwd

'/content'

In [9]:
ls

ml-20m/  ml-20m.zip  sample_data/


In [10]:
import pandas as pd

# 訓練データとテストデータを分けるのに使用する
from sklearn import model_selection

df = pd.read_csv("/content/ml-20m/ratings.csv")

# Xは(userID, movieId)のペア
X = df[["userId", "movieId"]].values
X

array([[     1,      2],
       [     1,     29],
       [     1,     32],
       ...,
       [138493,  69644],
       [138493,  70286],
       [138493,  71619]])

In [11]:
Y = df[["rating"]].values
Y

array([[3.5],
       [3.5],
       [3.5],
       ...,
       [3. ],
       [5. ],
       [2.5]])

In [0]:
# 訓練データとテストデータを9:1に分割
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(X, Y, test_size=0.1)

In [13]:
train_X.shape

(18000236, 2)

In [14]:
test_X.shape

(2000027, 2)

In [15]:
train_Y.shape

(18000236, 1)

In [16]:
test_Y.shape

(2000027, 1)

In [0]:
# XはIDで整数なのでint64、Yは実数値なのでfloat32のTensorに変換する
train_dataset = TensorDataset(torch.tensor(train_X, dtype=torch.int64),
                               torch.tensor(train_Y, dtype=torch.float32))

In [18]:
train_dataset.tensors

(tensor([[125469,   1208],
         [ 10477,   1826],
         [123352,  81516],
         ...,
         [ 99519,   2352],
         [  1620,   2706],
         [ 62100,   1222]]), tensor([[5.0000],
         [1.0000],
         [3.5000],
         ...,
         [2.0000],
         [4.0000],
         [4.5000]]))

In [0]:
test_dataset = TensorDataset(torch.tensor(test_X, dtype=torch.int64),
                               torch.tensor(test_Y, dtype=torch.float32))

In [20]:
test_dataset.tensors

(tensor([[132616,   2020],
         [ 17572,   4878],
         [ 29575,   3130],
         ...,
         [  6636,   3571],
         [ 15189,    163],
         [ 75880,    273]]), tensor([[2.],
         [5.],
         [2.],
         ...,
         [4.],
         [4.],
         [4.]]))

In [0]:
train_loader = DataLoader(train_dataset, batch_size=1024, num_workers=4,shuffle=True)

In [22]:
print(train_loader.batch_size)
print(train_loader.num_workers)
print(train_loader.dataset)

1024
4


In [23]:
train_loader.dataset.tensors

(tensor([[125469,   1208],
         [ 10477,   1826],
         [123352,  81516],
         ...,
         [ 99519,   2352],
         [  1620,   2706],
         [ 62100,   1222]]), tensor([[5.0000],
         [1.0000],
         [3.5000],
         ...,
         [2.0000],
         [4.0000],
         [4.5000]]))

In [0]:
test_loader = DataLoader(test_dataset, batch_size=1024,num_workers=4)

In [0]:
　# PyTorrchで行列因子分解

class MatrixFactorization(nn.Module):
  def __init__(self, max_user, max_item, k=20):
    super().__init__()
    self.max_user = max_user
    self.max_item = max_item
    self.user_emb = nn.Embedding(max_user, k, 0)
    self.item_emb = nn.Embedding(max_item, k, 0)

  def forward(self, x):
    user_idx = x[:, 0]
    item_idx = x[:, 1]
    user_feature = self.user_emb(user_idx)
    item_feature = self.item_emb(item_idx)

    # user_feature*item_featureは(batch_size, k)次元なのでkについてsumを取るとそれぞれのサンプルの内積になる
    out = torch.sum(user_feature*item_feature, 1)

    # [0, 5]の範囲に収まるようにシグモイド関数を使って変換
    out = nn.functional.sigmoid(out)*5
    return out

In [0]:
# ユーザーや商品の個数
max_user, max_item = X.max(0)

# np.int64型をPythonの標準のintにキャスト
max_user = int(max_user)
max_item = int(max_item)
net = MatrixFactorization(max_user+1, max_item+1)

In [0]:
# 評価関数の作成(L1損失でMeanAbsoluteErrorを計算)
def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
  ys = []
  ypreds = []
  for x, y in loader:
    x = x.to(device)
    ys.append(y)
    with torch.no_grad():
      ypred = net(x).to("cpu").view(-1)
    ypreds.append(ypred)
  score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
  return score.item()

In [32]:
# 訓練部分の作成

from statistics import mean

net.to("cuda:0")
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

for epoch in range(5):
  loss_log = []
  for x, y, in tqdm.tqdm(train_loader):
    x = x.to("cuda:0")
    y = y.to("cuda:0")
    o = net(x)

    # 損失関数の計算
    loss = loss_f(o, y.view(-1))
    # 勾配をゼロにする
    net.zero_grad()
    # 誤差逆伝播法を使う
    loss.backward()
    # 更新する
    opt.step()
    # 損失関数に追加
    loss_log.append(loss.item())
  # epochごとに評価
  test_score = eval_net(net, test_loader, device="cuda:0")
  print(epoch, mean(loss_log), test_score, flush=True)

100%|██████████| 17579/17579 [02:41<00:00, 109.07it/s]


0 1.6111447886094494 0.7324641942977905


100%|██████████| 17579/17579 [02:45<00:00, 106.49it/s]


1 0.8787306211827334 0.7078908085823059


100%|██████████| 17579/17579 [02:44<00:00, 107.10it/s]


2 0.834682711043912 0.699458658695221


100%|██████████| 17579/17579 [02:44<00:00, 88.21it/s] 


3 0.8134696860790551 0.6958974003791809


100%|██████████| 17579/17579 [02:44<00:00, 106.60it/s]


4 0.8012945498953067 0.6939730644226074


In [44]:
# 訓練したモデルをCPUに移す
net.to("cpu")

# ユーザー1の映画10の評価を計算したい
query = (1, 10.0)

# int64のTensorに変換し、batchの次元を付与
query = torch.tensor(query, dtype=torch.int64).view(1, -1)

# netに渡す
net(query)

tensor([3.9927], grad_fn=<MulBackward0>)

In [0]:
# ユーザーの上位5本の映画をピックアップ
query = torch.stack([
                      torch.zeros(max_item).fill_(1),
                      torch.arange(1, max_item+1).float() # float()をつけたら動いた。
], 1).long()

# scoresは上位k本のスコア
# indicesは上位k本の位置、すなわちmovieId
scores, indices = torch.topk(net(query), 5)

In [50]:
scores

tensor([4.9999, 4.9998, 4.9998, 4.9998, 4.9998], grad_fn=<TopkBackward>)

In [51]:
indices

tensor([117301,   9173,  37305,  98864,  30002])

## ニューラル行列因子分解

In [0]:
# 非線形の関数を利用したモデリング
class NeuralMatrixFactorization(nn.Module):
  def __init__(self, max_user, max_item,
               user_k=10, item_k=10,
               hidden_dim=50):
    super().__init__()
    self.user_emb = nn.Embedding(max_user, user_k, 0)
    self.item_emb = nn.Embedding(max_item, item_k, 0)
    self.mlp = nn.Sequential(
        nn.Linear(user_k + item_k, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, 1)
    )

  def forward(self, x):
    user_idx = x[:, 0]
    item_idx = x[:, 1]
    user_feature = self.user_emb(user_idx)
    item_feature = self.item_emb(item_idx)

    # ユーザー特徴量と商品特徴量をまとめて1つのベクトルにする
    out = torch.cat([user_feature, item_feature], 1)

    # まとめた特徴量ベクトルをMLPに入れる
    out = self.mlp(out)
    out = nn.functional.sigmoid(out)*5
    return out.squeeze()

In [0]:
# 付属情報の利用
# 区切られたジャンルをBoWに変換

import csv
from sklearn.feature_extraction.text import CountVectorizer

# csv.DictReaderを使用してCSVファイルを読み込み
# 必要な部分だけ抽出する
with open('/content/ml-20m/movies.csv') as fp:
  reader = csv.DictReader(fp)
  def parse(d):
    movieId = int(d["movieId"])
    genres = d["genres"]
    return movieId, genres
  data = [parse(d) for d in reader]

# 内包表記で各要素ごとにデータを抽出
movieIds = [x[0] for x in data]
genres = [x[1] for x in data]

# データに合わせてCountVectorizerを訓練する
cv = CountVectorizer(dtype="f4").fit(genres)
num_genres = len(cv.get_feature_names())

# keyがmovieIdでvalueがBoWのTensorのdictを作る
it = cv.transform(genres).toarray()
it = (torch.tensor(g, dtype=torch.float32) for g in it)
genre_dict = dict(zip(movieIds, it))

In [0]:
# カスタムDatasetの作成
def first(xs):
  it = iter(xs)
  return next(it)

class MovieLensDataset(Dataset):
  def __init__(self, x, y, genres):
    # Assert statements are a convenient way to insert debugging assertions into a program. 
    assert len(x) == len(y)
    self.x = x
    self.y = y
    self.genres = genres

    # ジャンル辞書にないmovieIdのときのダミーデータ
    self.null_genre = torch.zeros_like(first(genres.values()))

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    x = self.x[idx]
    y = self.y[idx]
    # x = (userId, movieId)
    movieId = x[1]
    g = self.genres.get(movieId, self.null_genre)
    return x, y, g

In [0]:
# DataLoaderの作成

train_dataset = MovieLensDataset(
    torch.tensor(train_X, dtype=torch.int64),
    torch.tensor(train_Y, dtype=torch.float32),
    genre_dict
)


test_dataset = MovieLensDataset(
    torch.tensor(test_X, dtype=torch.int64),
    torch.tensor(test_Y, dtype=torch.float32),
    genre_dict
)


train_loader = DataLoader(
    train_dataset,
    batch_size=1024,
    shuffle=True,
    num_workers=4
)


test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=4
)




In [0]:
# ネットワークモデルの作成

class NeuralMatrixFactorization2(nn.Module):
  def __init__(self, max_user, max_item, num_genres,
               user_k = 10, item_k = 10, hidden_dim=50):
    super().__init__()
    self.user_emb = nn.Embedding(max_user, user_k, 0)
    self.item_emb = nn.Embedding(max_item, item_k, 0)
    self.mlp = nn.Sequential(
        # num_genres分だけ次元が増える
        nn.Linear(user_k + item_k + num_genres, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, 1)
    )

  def forward(self, x, g):
    user_idx = x[:, 0]
    item_idx = x[:, 1]
    user_feature = self.user_emb(user_idx)
    item_feature = self.item_emb(item_idx)

    # ジャンルのBoWをcatで特徴ベクトルに結合する
    out = torch.cat([user_feature, item_feature, g], 1)
    out = self.mlp(out)
    out = nn.functional.sigmoid(out)*5
    return out.squeeze()

In [0]:
# ヘルパー関数の修正

def eval_net(net, loader, score_fn = nn.functional.l1_loss, device="cpu"):
  ys = []
  ypreds = []
  # loaderはジャンルのBoWも返す
  for x, y, g in loader:
    x = x.to(device)
    g = g.to(device)
    ys.append(y)

    # userId, movieId以外にジャンルのBoWもネットワーク関数に渡す
    with torch.no_grad():
      ypred = net(x, g).to("cpu")
    ypreds.append(ypred)
  score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
  return score

In [0]:
# 訓練部分の作成

net = NeuralMatrixFactorization2(max_user+1, max_item+1, num_genres)
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

net.to("cuda:0")
for epoch in range(5):
  loss_log = []
  net.train()
  for x, y, g in tqdm.tqdm(train_loader):
    x = x.to("cuda:0")
    y = y.to("cuda:0")
    g = g.to("cuda:0")
    o = net(x, g)
    loss = loss_f(o, y.view(-1))
    net.zero_grad()
    loss.backward()
    opt.step()
    loss_log.append(loss.item())
  net.eval()
  test_score = eval_net(net, test_loader, device="cuda:0")
  print(epoch, mean(loss_log), test_score.item(), flush=True)

100%|██████████| 17579/17579 [03:49<00:00, 76.64it/s]


0 0.7583076585423479 0.6435149908065796


100%|██████████| 17579/17579 [03:51<00:00, 76.04it/s]


1 0.6924646805866538 0.6338335275650024


100%|██████████| 17579/17579 [03:53<00:00, 75.29it/s]


2 0.6712138898082816 0.6298700571060181


100%|██████████| 17579/17579 [03:53<00:00, 61.35it/s]


In [0]:
# 指定した位置だけ1で残りが0のTensorを返す補助関数

def make_genre_vector(i, max_len):
  g = torch.zeros(max_len)
  g[i] = 1
  return g

query_genres = [make_genre_vector(i, num_genres) for i in range(num_genres)]
query_genres = torch.stack(query_genres, 1)

# num_genres分だけuserId=100とmovieID=0のTensorを作って結合する
query = torch.stack([
                     torch.empty(num_genres, dtype=torch.int64).fill_(100),
                     torch.empty(num_genres, dtype=torch.int64).fill_(0)
], 1)

# GPUに転送
query_genres =query_genres.to("cuda:0")
query = query.to("cuda:0")

# スコアを計算
net(query, query_genres)